## Create trajectory

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ase.io import read
from ase.visualize.plot import plot_atoms

from ase.visualize import view

# Use the notebook backend for interactive plots
%matplotlib widget

In [2]:
import numpy as np

In [3]:
import json
from ase import Atoms
from ase.build import molecule
from ase.io import Trajectory, write

In [4]:
fe_top = [1.0, 0.0, 0.50]
fe_bottom = [0.69, 0.14, 0.36]
fe_climbing = [0.63, 0.58, 0.44]
fe_bridge = [0.7, 1.0, 0.44]
fe_trough = [0.59, 0.5, 0.31]

In [8]:
with open('coords 1e-3.txt') as f:
    molecular_positions = json.load(f)

molecular_positions = [[(molecular_position[0], molecular_position[1], molecular_position[2]),
                        (molecular_position[3], molecular_position[4], molecular_position[5]),
                        (molecular_position[6], molecular_position[7], molecular_position[8]),
                        fe_top, fe_bottom, fe_climbing, fe_bridge, fe_trough]
                       for molecular_position in molecular_positions]

Create traj file

In [6]:
# Example atomic symbols
symbols = ['N', 'H', 'H', 'Fe', 'Fe', 'Fe', 'Fe', 'Fe']

# Create a Trajectory object
traj = Trajectory('molecule 1e-3.traj', 'w')

# Loop through each frame and create an Atoms object
for frame in molecular_positions:
    atoms = Atoms(symbols=symbols, positions=frame)
    traj.write(atoms, rotation="22z")

# Close the trajectory file
traj.close()

## Animate trajectory

`ase gui molecule.traj`

In [19]:
size=(3,1,1)

In [26]:
from ase.build import bulk, surface
from ase.visualize import view

# Create bulk BCC iron
bulk_fe = bulk('Fe', 'bcc', a=2.856)  # lattice constant for BCC iron

# Create the (211) surface with 3 layers and 10 Å vacuum
fe_211_surface = surface(bulk_fe, (2, 1, 1), layers=3, vacuum=10.0)

# Center the surface slab in the cell
fe_211_surface.center(axis=2)

# View the structure
view(fe_211_surface, viewer='ngl')

In [16]:
#coords = [-4.3857516, 2.42202895, -4.41477892, -3.88343244,  2.62167029, -4.71856799, -4.65836314,  2.78615741, -3.9935248]

fe_top = [1.0, 0.0, 0.50]
fe_bottom = [0.69, 0.14, 0.36]
fe_climbing = [0.63, 0.58, 0.44]
fe_bridge = [0.7, 1.0, 0.44]
fe_trough = [0.59, 0.5, 0.31]


fe_lattice = Atoms(
 #   f"FeFeFeFeFeNHH",
    f"FeFeFeFeFe",
    np.concatenate(
        [
            [fe_top, fe_bottom, fe_climbing, fe_bridge, fe_trough],
           # np.reshape(coords, (-1, 3)),
        ],        
    ),
)

view(fe_lattice, viewer='ngl')

In [46]:
from ase import Atoms
from ase.build import fcc211, bulk
from ase.io import read, write

a = 2.586  # vertex constant

slab = fcc211('Fe', (3, 2, 3), a=a)
view(slab, viewer='ngl')

In [47]:
slab.get_positions()

array([[ 2.98605559e+00,  9.14289068e-01,  4.22292032e+00],
       [ 2.98605559e+00,  2.74286720e+00,  4.22292032e+00],
       [ 1.49302780e+00, -1.43551837e-16,  3.69505528e+00],
       [ 1.49302780e+00,  1.82857814e+00,  3.69505528e+00],
       [ 1.25981645e-16,  9.14289068e-01,  3.16719024e+00],
       [ 1.25981645e-16,  2.74286720e+00,  3.16719024e+00],
       [ 2.98605559e+00, -1.40760964e-16,  2.63932520e+00],
       [ 2.98605559e+00,  1.82857814e+00,  2.63932520e+00],
       [ 1.49302780e+00,  9.14289068e-01,  2.11146016e+00],
       [ 1.49302780e+00,  2.74286720e+00,  2.11146016e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.58359512e+00],
       [ 0.00000000e+00,  1.82857814e+00,  1.58359512e+00],
       [ 2.98605559e+00,  9.14289068e-01,  1.05573008e+00],
       [ 2.98605559e+00,  2.74286720e+00,  1.05573008e+00],
       [ 1.49302780e+00, -1.43551837e-16,  5.27865040e-01],
       [ 1.49302780e+00,  1.82857814e+00,  5.27865040e-01],
       [ 1.25981645e-16,  9.14289068e-01